In [1]:
!pip install google-generativeai

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ------- -------------------------------- 2.4/13.3 MB 13.4 MB/s eta 0:00:01
   --------------- ------------------------ 5.2/13.3 MB 12.7 MB/s eta 0:00:01
   ----------------------- ---------------- 7.9/13.3 MB 12.8 MB/s eta 0:00:01
   ----------------------------- ---------- 10.0/13.3 MB 12.2 MB/s eta 0:00:01
   -------------------------------------- - 12.8/13.3 MB 12.4 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 11.8 MB/s eta 0:00:00


In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import google.generativeai as genai
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# --------------- Gemini 설정 ---------------
genai.configure(api_key="YOUR_GEMINI_API_KEY")
model = genai.GenerativeModel('gemini-pro')

# 회의 내용 예시
meeting_transcript = """
오늘 회의에서는 마케팅 캠페인 진행 상황을 공유했습니다.
- 유튜브 영상은 다음 주 업로드 예정
- 블로그 콘텐츠는 초안 완료
- SNS 채널별 일정 조정 필요
또한 다음 분기 예산안을 검토했고, 마케팅 비용 조정을 요청하기로 했습니다.
"""

# 회의 요약
def summarize_meeting(transcript):
    prompt = f"""
다음 회의록을 간단히 요약해 주세요:

{transcript}
"""
    response = model.generate_content(prompt)
    return response.text.strip()

# 행동 항목 추출
def extract_action_items(transcript):
    prompt = f"""
다음 회의록에서 수행해야 할 행동 항목을 뽑아 주세요. 각 항목은 리스트 형태로 출력해 주세요:

{transcript}
"""
    response = model.generate_content(prompt)
    return [line.strip("- ").strip() for line in response.text.splitlines() if line.strip()]

# Google Tasks API 인증 및 서비스 생성
def get_tasks_service():
    SCOPES = ['https://www.googleapis.com/auth/tasks']
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('tasks', 'v1', credentials=creds)
    return service

# Google Tasks에 할 일 추가
def upload_to_google_tasks(tasks, service, tasklist_id='@default'):
    for task in tasks:
        task_body = {'title': task}
        service.tasks().insert(tasklist=tasklist_id, body=task_body).execute()
        print(f"✅ 업로드 완료: {task}")

# ---------------- 실행 ----------------

summary = summarize_meeting(meeting_transcript)
action_items = extract_action_items(meeting_transcript)

print("📄 회의 요약:\n", summary)
print("\n📝 행동 항목:\n", "\n".join(f"- [ ] {item}" for item in action_items))

# Google Tasks로 업로드
print("\n🚀 Google Tasks로 업로드 중...")
service = get_tasks_service()
upload_to_google_tasks(action_items, service)
